In [1]:
from Datasets.ImageNetDataset import ImageNet100
from train_hae import load_hae_from_checkpoints

from torchvision import transforms
import torchvision.models as models
import torch 
import torch.nn as nn
from torchinfo import summary
ROOT_FOLDER = '/home/dl_class/data/ILSVRC/Data/CLS-LOC/'

model = load_hae_from_checkpoints()
imageNet100 = ImageNet100(ROOT_FOLDER, split='val', transform=transforms.Compose([
                                                            transforms.CenterCrop(224),
                                                            transforms.ToTensor()]))

print(f'Loaded {len(model)} layer model')

Loaded 5 layer model


In [2]:
print([name for name, _ in model[0].named_children()])

['encoder', 'decoder', 'normalize']


In [3]:
from models.hae import Encoder
encoder = Encoder(in_feat_dim=3, codebook_dim=64, hidden_dim=16, num_res_blocks=0)
enc_dict = encoder.state_dict()
pretrained_dict = model[0].encoder.state_dict()
print(pretrained_dict.items())
pre_dict = {k: v for k, v in pretrained_dict.items() if k in enc_dict}

odict_items([('blocks.0.weight', tensor([[[[ 1.8449e-01, -1.1478e-01,  1.0730e-01],
          [ 2.0714e-01,  2.8318e-01,  1.4811e-01],
          [-2.9361e-02,  1.1176e-01,  1.6015e-01]],

         [[ 2.4140e-01,  1.3538e-01, -1.3770e-01],
          [ 2.2994e-01,  1.6239e-01,  9.6886e-02],
          [-1.3221e-01,  1.8116e-01, -2.3238e-02]],

         [[ 1.8690e-01, -5.4901e-02,  3.9920e-02],
          [-1.5778e-01, -1.9448e-01, -1.4815e-01],
          [-2.4506e-01, -1.7711e-01,  1.0724e-01]]],


        [[[-8.9761e-02,  1.7940e-01,  1.5724e-01],
          [ 8.8407e-02,  4.6130e-01,  1.4884e-01],
          [ 4.1392e-02,  2.4983e-01,  1.6854e-01]],

         [[-8.0987e-02,  8.1577e-02, -1.0960e-01],
          [-4.9472e-02,  3.1404e-01, -7.7628e-03],
          [-2.0415e-01, -4.7653e-02, -1.6337e-01]],

         [[ 8.4837e-02,  4.0692e-01, -1.4585e-02],
          [-1.3545e-01,  3.1863e-01,  1.7282e-01],
          [-2.2287e-01, -2.0003e-01, -1.8991e-02]]],


        [[[ 8.6797e-02, -6.9792e-

In [5]:
print(pre_dict)

{'blocks.0.weight': tensor([[[[ 1.8449e-01, -1.1478e-01,  1.0730e-01],
          [ 2.0714e-01,  2.8318e-01,  1.4811e-01],
          [-2.9361e-02,  1.1176e-01,  1.6015e-01]],

         [[ 2.4140e-01,  1.3538e-01, -1.3770e-01],
          [ 2.2994e-01,  1.6239e-01,  9.6886e-02],
          [-1.3221e-01,  1.8116e-01, -2.3238e-02]],

         [[ 1.8690e-01, -5.4901e-02,  3.9920e-02],
          [-1.5778e-01, -1.9448e-01, -1.4815e-01],
          [-2.4506e-01, -1.7711e-01,  1.0724e-01]]],


        [[[-8.9761e-02,  1.7940e-01,  1.5724e-01],
          [ 8.8407e-02,  4.6130e-01,  1.4884e-01],
          [ 4.1392e-02,  2.4983e-01,  1.6854e-01]],

         [[-8.0987e-02,  8.1577e-02, -1.0960e-01],
          [-4.9472e-02,  3.1404e-01, -7.7628e-03],
          [-2.0415e-01, -4.7653e-02, -1.6337e-01]],

         [[ 8.4837e-02,  4.0692e-01, -1.4585e-02],
          [-1.3545e-01,  3.1863e-01,  1.7282e-01],
          [-2.2287e-01, -2.0003e-01, -1.8991e-02]]],


        [[[ 8.6797e-02, -6.9792e-03,  4.0716e-

In [6]:
enc_dict.update(pre_dict)
encoder.load_state_dict(enc_dict)

<All keys matched successfully>

In [7]:
x = torch.rand(1, 3, 224, 224)

In [8]:
body = encoder
modules = []
for child in body.children():
    if type(child)==nn.Sequential:
        for layer in child.children():
            modules.append(layer)
    else:
        modules.append(child)
modules.append(nn.ReLU())
modules.append(nn.AdaptiveAvgPool2d((1,1)))
modules.append(nn.Flatten())
bmodel = nn.Sequential(*modules)
y = bmodel(x)
summary(bmodel, input_size=(1, 3, 224, 224), row_settings=("depth", "ascii_only"))
        


Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 64]                   --
+ Conv2d: 1-1                            [1, 8, 112, 112]          224
+ Mish: 1-2                              [1, 8, 112, 112]          --
+ Conv2d: 1-3                            [1, 16, 112, 112]         1,168
+ Mish: 1-4                              [1, 16, 112, 112]         --
+ Conv2d: 1-5                            [1, 64, 112, 112]         1,088
+ ReLU: 1-6                              [1, 64, 112, 112]         --
+ AdaptiveAvgPool2d: 1-7                 [1, 64, 1, 1]             --
+ Flatten: 1-8                           [1, 64]                   --
Total params: 2,480
Trainable params: 2,480
Non-trainable params: 0
Total mult-adds (M): 31.11
Input size (MB): 0.60
Forward/backward pass size (MB): 8.83
Params size (MB): 0.01
Estimated Total Size (MB): 9.44

In [9]:
y.shape

torch.Size([1, 64])

In [10]:
print(body)

Encoder(
  (blocks): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): Mish()
    (2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): Mish()
    (4): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
  )
)
